In [9]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "xgboost"])

0

In [10]:
import sys
print(sys.executable)

c:\Users\jiamv\miniconda3\python.exe


In [15]:
import pandas as pd 
import xgboost 
df = pd.read_excel(r'C:\Users\jiamv\Downloads\Low_Conc_Experiments.xlsx')

x = df.iloc[:, [1] + list(range(3, 7))]
y = df.iloc[:, list(range(1, 3))]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

from xgboost import XGBRegressor

xg = XGBRegressor(n_estimators=100, random_state=42)
xg.fit(X_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,None
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes

In [29]:
x.head(2)

,Time in minutes,Honeywell_01,Honeywell_03,Honeywell_04,Plantower_01
0,1,160.666667,288.666667,160.500000,146.444444
1,2,159.125000,283.500000,156.333333,142.375000


In [32]:
df.head(100)

,date & time,Time in minutes,Grimm,Honeywell_01,Honeywell_03,Honeywell_04,Plantower_01,Plantower_03,Plantower_04,SPS30_01,SPS30_03,SPS30_04
0,2025-03-25 10:48:00,1,153.92,160.666667,288.666667,160.500000,146.444444,302.666667,163.875000,155.182222,276.164444,139.057500
1,2025-03-25 10:49:00,2,153.27,159.125000,283.500000,156.333333,142.375000,302.500000,163.555556,151.732500,271.147500,131.615556
2,2025-03-25 10:50:00,3,151.28,168.222222,279.555556,164.111111,152.555556,291.555556,182.555556,160.726667,262.652222,142.931111
3,2025-03-25 10:51:00,4,154.71,222.222222,274.666667,244.555556,238.555556,298.444444,282.000000,239.026667,259.520000,241.676667
4,2025-03-25 10:52:00,5,195.83,280.111111,266.888889,270.625000,308.555556,294.666667,322.625000,299.777778,256.345556,271.020000
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-03-25 12:23:00,96,44.73,49.555556,35.333333,38.111111,50.444444,47.222222,56.555556,35.557778,31.025556,31.465556
96,2025-03-25 12:24:00,97,43.74,47.666667,34.777778,36.000000,49.444444,45.666667,51.888889,34.228889,29.915556,31.753333
97,2025-03-25 12:25:00,98,42.34,45.777778,33.333333,35.111111,48.888889,44.666667,53.555556,32.506667,28.111111,30.547778
98,2025-03-25 12:26:00,99,40.76,45.500000,32.125000,34.625000,47.750000,43.375000,52.000000,31.276250,27.725000,29.607500


In [27]:
y_pred = xg.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)
y_pred_df['Time'] = df['Time in minutes']
y_pred_df 

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared Score:", r2)
print(xg.feature_importances_)

Mean Absolute Error: 2.194955825805664
Mean Squared Error: 11.09326171875
R-squared Score: 0.997178316116333
[0.90736246 0.03661913 0.00431325 0.01119524 0.04050998]


In [ ]:
# Use full dataset or a large subset
X_distill = x.copy()

# XGBoost predictions (teacher outputs)
y_distill = xg.predict(X_distill)


In [26]:
y_distill.shape

(171, 2)

In [ ]:
# Split outputs
y1_distill = y_distill[:, 0]
y2_distill = y_distill[:, 1]

In [21]:
from sklearn.ensemble import RandomForestRegressor

student_1 = RandomForestRegressor(
    n_estimators=8,
    max_depth=3,
    random_state=42
)

student_2 = RandomForestRegressor(
    n_estimators=8,
    max_depth=3,
    random_state=42
)

student_1.fit(X_distill, y1_distill)
student_2.fit(X_distill, y2_distill)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",8
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",3
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at 

In [ ]:
y1_student_pred = student_1.predict(X_test)
y2_student_pred = student_2.predict(X_test)

from sklearn.metrics import r2_score

print("Output 1 R2:", r2_score(y_test.iloc[:,0], y1_student_pred))
print("Output 2 R2:", r2_score(y_test.iloc[:,1], y2_student_pred))
print(student_1.feature_importances_)
print(student_2.feature_importances_)


Output 1 R2: 0.9936483685213938
Output 2 R2: 0.9923058580396008
[0.53072704 0.10466371 0.232698   0.11617823 0.01573303]
[0.37771308 0.12129853 0.35784757 0.12664411 0.01649671]


In [23]:
import micromlgen
from micromlgen import port

with open("model_output1.h", "w") as f:
    f.write(port(student_1))

with open("model_output2.h", "w") as f:
    f.write(port(student_2))